In [40]:
import base64
import dotenv
import requests
import os
import pandas as pd 

dotenv.load_dotenv()

OCTOPUS_API_KEY = os.getenv("OCTOPUS_API_KEY")
OCTOPUS_BASE_URL = "https://api.octopus.energy/v1"
ELECTRICITY_MPAN = os.getenv("METER_MPAN")
ELECTRICITY_SERIAL = os.getenv("METER_SERIAL")
GAS_MPRN = os.getenv("GAS_MPRN")  # Add this to your .env file
GAS_SERIAL = os.getenv("GAS_SERIAL")  # Add this to your .env file

periodFrom = "2023-01-01T00:00:00Z"
periodTo = "2024-11-20T00:00:00Z"

def fetch_consumption(url):
    results = []
    # Base 64 encode the API key
    b64_api_key = base64.b64encode(OCTOPUS_API_KEY.encode())
    
    while True:
        response = requests.get(url, headers={"Authorization": f"Basic {b64_api_key.decode()}:"})
        response_json = response.json()
        results.extend(response_json["results"])

        next_url = response_json.get("next")
        if next_url:
            print("got next...")
            url = response_json["next"]
        else:
            break

    df = pd.DataFrame(results)
    df['interval_start'] = pd.to_datetime(df['interval_start'], utc=True)
    df['interval_end'] = pd.to_datetime(df['interval_end'], utc=True)
    return df 

# Fetch electricity consumption
electricity_url = f"{OCTOPUS_BASE_URL}/electricity-meter-points/{ELECTRICITY_MPAN}/meters/{ELECTRICITY_SERIAL}/consumption/?period_from={periodFrom}&period_to={periodTo}&group_by=day"
electricity = fetch_consumption(electricity_url)

# Fetch gas consumption
gas_url = f"{OCTOPUS_BASE_URL}/gas-meter-points/{GAS_MPRN}/meters/{GAS_SERIAL}/consumption/?period_from={periodFrom}&period_to={periodTo}&group_by=day"
gas = fetch_consumption(gas_url)

print(f"Fetched {len(electricity)} electricity readings")
print(f"Fetched {len(gas)} gas readings")

got next...
got next...
got next...
got next...
got next...
got next...
got next...
got next...
got next...
got next...
got next...
got next...
Fetched 690 electricity readings
Fetched 667 gas readings


In [82]:
import pandas as pd

# Constants for costs
ELECTRICITY_STANDING_CHARGE = 0.4879  # in £/day
ELECTRICITY_UNIT_RATE = 0.2425  # in £/kWh
GAS_STANDING_CHARGE = 0.2938  # in £/day
GAS_UNIT_RATE = 0.0585  # in £/kWh
GAS_M3_TO_KWH = 11.104  # conversion factor from m³ to kWh

# Convert results to pandas DataFrames
electric = pd.DataFrame(electricity)
gas = pd.DataFrame(gas)

# Convert dates
electric['date'] = pd.to_datetime(electric['interval_start']).dt.date
gas['date'] = pd.to_datetime(gas['interval_start']).dt.date

# Set the date as index
electric.set_index('date', inplace=True)
gas.set_index('date', inplace=True)

# Convert gas consumption from m³ to kWh
gas['consumption_kwh'] = gas['consumption'] * GAS_M3_TO_KWH

# Calculate costs
electric['electricity_cost'] = (electric['consumption'] * ELECTRICITY_UNIT_RATE) + ELECTRICITY_STANDING_CHARGE
gas['gas_cost'] = (gas['consumption_kwh'] * GAS_UNIT_RATE) + GAS_STANDING_CHARGE

# Merge the two dataframes
df = pd.merge(
    electric[['consumption', 'electricity_cost']], 
    gas[['consumption', 'consumption_kwh', 'gas_cost']], 
    left_index=True, 
    right_index=True,
    suffixes=('_electricity', '_gas_m3'),
    how='outer'
)

# Calculate total daily cost
df['total_cost'] = df['electricity_cost'] + df['gas_cost'].fillna(0)


# Get weather data

In [83]:
import requests
import pandas as pd
from datetime import datetime

# Hitchin coordinates
LATITUDE = 51.95
LONGITUDE = -0.28

# Convert the period dates to the format OpenMeteo expects (YYYY-MM-DD)
start_date = pd.to_datetime(periodFrom).strftime('%Y-%m-%d')
end_date = pd.to_datetime(periodTo).strftime('%Y-%m-%d')

# OpenMeteo API URL
url = f"https://archive-api.open-meteo.com/v1/archive"

# Parameters for the API request
params = {
    'latitude': LATITUDE,
    'longitude': LONGITUDE,
    'start_date': start_date,
    'end_date': end_date,
    'daily': 'temperature_2m_mean',  # daily average temperature
    'timezone': 'Europe/London'
}

# Make the request
response = requests.get(url, params=params)
weather_data = response.json()

# Convert to DataFrame
weather = pd.DataFrame({
    'date': pd.to_datetime(weather_data['daily']['time']).date,
    'temperature': weather_data['daily']['temperature_2m_mean']
})

# Set date as index to match our existing dataframe
weather.set_index('date', inplace=True)

# Merge with our existing combineb

In [87]:
import plotly.express as px

tmp = (
    df.merge(weather, left_index=True, right_index=True)
    .reset_index()

    .assign(
        date=lambda x: pd.to_datetime(x["date"]),
        pre=lambda x: x['date'] < pd.to_datetime("2024-10-28"),
        post=lambda x: x['date'] >= pd.to_datetime("2024-11-01"),
        time=lambda x: x.pre.replace({True: "pre", False: "post"}),
    )
    .loc[lambda x: x["pre"] | x["post"]]
    .reset_index()
    .pipe(
        px.scatter,
        x="temperature",
        y="total_cost",
        color="time",
    )
)

tmp